<a href="https://colab.research.google.com/github/golf41101/Suchet/blob/master/Recommendation_user_user_cosineSimilality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import seaborn as sns
import os
from scipy.spatial.distance import cosine
import matplotlib.pyplot as plt

In [2]:
import warnings

warnings.filterwarnings('ignore')

In [3]:
pd.set_option('display.max_columns', 20)

## Load Data

In [4]:
from google.colab import drive
#mount you GoogleDrive to "drive" folder
drive.mount('/drive', force_remount=False)

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [5]:
#set path
path ="/drive/MyDrive/Recommendation/"
os.chdir(path)

In [6]:
!ls

movie.csv  rating.csv  ratings_small.csv


In [7]:
rating = pd.read_csv(path + "ratings_small.csv")

In [8]:
movie = pd.read_csv(path + "movie.csv")

In [9]:
movie.drop(columns='genres',inplace=True)

In [10]:
df = rating.merge(movie,on='movieId')

In [11]:
df.drop(columns='timestamp',inplace=True)

## Data Understanding

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99343 entries, 0 to 99342
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   userId   99343 non-null  int64  
 1   movieId  99343 non-null  int64  
 2   rating   99343 non-null  float64
 3   title    99343 non-null  object 
dtypes: float64(1), int64(2), object(1)
memory usage: 3.8+ MB


In [13]:
df.isnull().sum()

userId     0
movieId    0
rating     0
title      0
dtype: int64

In [14]:
df.head()

,userId,movieId,rating,title
0,1,31,2.5,Dangerous Minds (1995)
1,7,31,3.0,Dangerous Minds (1995)
2,31,31,4.0,Dangerous Minds (1995)
3,32,31,4.0,Dangerous Minds (1995)
4,36,31,3.0,Dangerous Minds (1995)


## EDA

In [15]:
print(df['userId'].unique())

print(df['userId'].nunique())

[  1   7  31  32  36  39  73  88  96 110 111 150 161 165 186 242 254 288
 310 311 325 338 341 386 394 452 457 461 468 485 487 496 511 516 525 590
 596 601 607 619 639 641  19  35  61  99 102 128 175 185 188 213 214 268
 294 303 363 387 420 442 466 471 472 514 518 529 530 547 551 564 575 577
 582 602 603 605 646 665  23  30  70  90 105 119 141 212 233 252 292 306
 318 380 507 509 534 537 584 608 647  21  34  41  49  57  66  77 103 118
 155 177 195 248 276 299 346 351 353 358 390 423 463 522 580 585 597 624
 634 654  38  56  94 130 133 148 198 229 232 262 280 283 320 321 330 372
 373 391 430 441 460 481 497 505 510 521 539 545 587  15  17  22 152 257
 265 309 344 370 434 527 533 592 648 661 137 207 313 326 335 388 404 405
 470 488 559 598  20  48  78 211 236 281 419 532 576  81  95 243 247 285
 290 297 300 308 312 337 362 403 560 562 614 617 618 658 664  27 120 291
 342 407 613   4 202 217 439 458 477 546 574  25  26  87 154 224 361 384
 428 502 586 606 220 360 431 436 520 594 669  59 19

userID is [1,138493] : n,n+1,...

In [16]:
print(df['movieId'].unique())

print(df['movieId'].nunique())

[  31 1029 1061 ...  129 4736 6425]
8794


movieID is not sereis number

### Give userId and movieId start to 0 and rearrange movieId [0,N-1]

In [17]:
# rating['userId'] = rating['userId'] - 1

In [18]:
# print(rating['userId'].unique())

# print(rating['userId'].nunique())

In [19]:
# rating['movieId'].values

In [20]:
# # create a mapping for movie ids
# unique_movie_ids = set(rating['movieId'].values)
# movie2idx = {}
# count = 0
# for movie_id in unique_movie_ids:
#   movie2idx[movie_id] = count
#   count += 1

In [21]:
# rating['movie_idx'] = rating.apply(lambda row: movie2idx[row.movieId], axis=1)

In [22]:
# from surprise import accuracy, SVD,Reader,Dataset
# from surprise.model_selection import train_test_split

# reader = Reader()
# df.head()

In [23]:
# it take so much rams,then i select 100000 data to this tasks

In [24]:
df.shape

(99343, 4)

In [26]:
rating_small = df.copy()

In [27]:
rating_small

,userId,movieId,rating,title
0,1,31,2.5,Dangerous Minds (1995)
1,7,31,3.0,Dangerous Minds (1995)
2,31,31,4.0,Dangerous Minds (1995)
3,32,31,4.0,Dangerous Minds (1995)
4,36,31,3.0,Dangerous Minds (1995)
...,...,...,...,...
99338,664,64997,2.5,War of the Worlds (2005)
99339,664,72380,3.5,"Box, The (2009)"
99340,665,129,3.0,Pie in the Sky (1996)
99341,665,4736,1.0,Summer Catch (2001)


## User-Based Filtering 

In [31]:
rating_mt = rating_small.pivot_table(index='userId', columns='title', values='rating').fillna(0)

In [32]:
rating_mt

title,$9.99 (2008),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (1964),Zulu (2013),[REC] (2007),"\\""Great Performances\""\"" Cats (1998)""",eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
669,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
rating_mt.shape

(671, 8792)

In [34]:
user_similarity = rating_mt.T
user_similarity

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
title,,,,,,,,,,,,,,,,,,,,,
$9.99 (2008),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Neath the Arizona Skies (1934),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
loudQUIETloud: A Film About the Pixies (2006),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
xXx (2002),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
xXx: State of the Union (2005),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
user_similarity_pe = rating_mt.T.corr()
user_similarity_pe.head()

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,-0.004088,-0.003382,0.068592,0.012130,-0.003000,0.079975,-0.005121,0.009729,-0.003184,...,-0.003569,-0.001275,0.003904,0.034915,-0.002887,-0.003890,-0.002046,0.060268,-0.002564,0.012486
2,-0.004088,1.000000,0.118387,0.107040,0.095056,-0.006000,0.206081,0.104139,0.107757,0.037129,...,0.473607,0.114344,0.058711,0.148962,0.463363,0.420996,0.080986,0.018613,0.166438,0.104144
3,-0.003382,0.118387,1.000000,0.071530,0.144867,0.056031,0.148613,0.243608,0.129954,0.110009,...,0.156254,0.116474,0.164754,0.146199,0.173178,0.118936,0.122003,0.076676,0.132965,0.163302
4,0.068592,0.107040,0.071530,1.000000,0.116884,0.070734,0.310253,0.177353,0.020320,0.128474,...,0.104086,0.083729,0.106203,0.230059,0.113918,0.077115,0.062513,0.096248,0.046678,0.198347
5,0.012130,0.095056,0.144867,0.116884,1.000000,0.057251,0.086573,0.155687,0.079900,0.025157,...,0.184321,0.036170,0.126319,0.207628,0.134014,0.049662,0.038407,0.031989,0.057075,0.215797


In [44]:
user_similarity_pe.shape

(671, 671)

In [36]:
# Create a placeholder dataframe listing user vs user
data_mv = pd.DataFrame(index=rating_mt.columns,columns=rating_mt.columns)

In [37]:
# User similarity matrix using cosine similarity
from sklearn.metrics.pairwise import cosine_similarity

In [46]:
rating_mt.head()

title,$9.99 (2008),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (1964),Zulu (2013),[REC] (2007),"\\""Great Performances\""\"" Cats (1998)""",eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
user_similarity_co = cosine_similarity(rating_mt)

In [48]:
user_similarity_co.shape

(671, 671)

In [49]:
user_similarity_co

array([[1.        , 0.        , 0.        , ..., 0.06291708, 0.        ,
        0.01746565],
       [0.        , 1.        , 0.12429498, ..., 0.02413984, 0.17059464,
        0.1131753 ],
       [0.        , 0.12429498, 1.        , ..., 0.08098382, 0.13660585,
        0.17019275],
       ...,
       [0.06291708, 0.02413984, 0.08098382, ..., 1.        , 0.04260878,
        0.08520194],
       [0.        , 0.17059464, 0.13660585, ..., 0.04260878, 1.        ,
        0.22867673],
       [0.01746565, 0.1131753 , 0.17019275, ..., 0.08520194, 0.22867673,
        1.        ]])

In [55]:
# fill data to array
user = pd.DataFrame(user_similarity_co, columns=rating_mt.T.columns,index=rating_mt.T.columns)

In [56]:
user

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.000000,0.000000,0.074482,0.016818,0.000000,0.083884,0.000000,0.012843,0.000000,...,0.000000,0.000000,0.014574,0.043719,0.000000,0.000000,0.000000,0.062917,0.000000,0.017466
2,0.000000,1.000000,0.124295,0.118821,0.103646,0.000000,0.212985,0.113190,0.113333,0.043213,...,0.477306,0.116373,0.078284,0.164162,0.466281,0.425462,0.084646,0.024140,0.170595,0.113175
3,0.000000,0.124295,1.000000,0.081640,0.151531,0.060691,0.154714,0.249781,0.134475,0.114672,...,0.161205,0.118207,0.177356,0.158357,0.177098,0.124562,0.124911,0.080984,0.136606,0.170193
4,0.074482,0.118821,0.081640,1.000000,0.130649,0.079648,0.319745,0.191013,0.030417,0.137186,...,0.114319,0.086959,0.137527,0.254030,0.121905,0.088735,0.068483,0.104309,0.054512,0.211609
5,0.016818,0.103646,0.151531,0.130649,1.000000,0.063796,0.095888,0.165712,0.086616,0.032370,...,0.191029,0.038929,0.147187,0.224245,0.139721,0.058252,0.042926,0.038358,0.062642,0.225086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,0.000000,0.425462,0.124562,0.088735,0.058252,0.000000,0.232051,0.069005,0.066412,0.032653,...,0.342283,0.093452,0.074595,0.124214,0.351207,1.000000,0.091597,0.018416,0.153111,0.127995
668,0.000000,0.084646,0.124911,0.068483,0.042926,0.019563,0.058773,0.112366,0.194493,0.098561,...,0.074089,0.108654,0.093667,0.082525,0.114487,0.091597,1.000000,0.000000,0.178017,0.135387
669,0.062917,0.024140,0.080984,0.104309,0.038358,0.024583,0.073151,0.055143,0.029291,0.060549,...,0.015960,0.047787,0.078468,0.101707,0.028773,0.018416,0.000000,1.000000,0.042609,0.085202


In [ ]:
# movie_watched = rating_mt["Seven (a.k.a. Se7en) (1995)"]
# similarity_with_other_movies = rating_mt.corrwith(movie_watched)

# similarity_with_other_movies = similarity_with_other_movies.sort_values(ascending=False)
# similarity_with_other_movies.head()

In [ ]:
# rating_mt.reset_index(drop=True,inplace=True)

In [ ]:
rating_mt = rating_mt.rename_axis(None, axis=1)

In [ ]:
rating_mt.columns

In [ ]:
rating_mt.head()

In [ ]:
# data_mv.reset_index(inplace=True)

In [ ]:
# data_mv = data_mv.rename_axis(None, axis=1)

In [ ]:
data_mv.head()

In [ ]:
data_mv.shape

In [ ]:
for i in range(0,len(data_mv.columns)) :
    # Loop through the columns for each column
    for j in range(0,len(data_mv.columns)) :
      # Fill in placeholder with cosine similarities
      data_mv.iloc[i,j] = 1-cosine(rating_mt.iloc[:,i],rating_mt.iloc[:,j])
      
data_mv

In [ ]:
tp_shape_data = data_mv.shape[0]

In [ ]:
data_mv.iloc[1,2]

In [ ]:
# Create a placeholder items for closes neighbours to an item
data_neighbours = pd.DataFrame(index=data_mv.columns,columns=range(1,12))

In [ ]:
data_neighbours

In [ ]:
# Loop through our similarity dataframe and fill in neighbouring item names
for i in range(0,len(data_mv.columns)):
  data_neighbours.iloc[i,:tp_shape_data] = data_mv.iloc[0:,i].sort_values(ascending=False)[:11].index
 
# --- End Item Based Recommendations --- #

In [ ]:
data_neighbours

In [ ]:
recommend = data_neighbours.iloc[:,1:]

In [ ]:
recommend.reset_index(inplace=True)

In [ ]:
recommend.head()

In [ ]:
recommend['index'].unique()

In [ ]:
recommend

In [ ]:
### reccomend movie like a 'Twelve Monkeys (a.k.a. 12 Monkeys) (1995)'
recommend[recommend['index']=='Twelve Monkeys (a.k.a. 12 Monkeys) (1995)']

In [ ]:
rec_wan = recommend.loc[recommend['index'].str.contains("Pay It Forward", case=False)]
rec_wan

In [ ]:
rec_wan

In [ ]:
# --- Start User Based Recommendations --- #
 
# Helper function to get similarity scores
def getScore(history, similarities):
   return sum(history*similarities)/sum(similarities)

In [ ]:
rating_mt_b = rating_small.pivot(index='userId',columns='title',values='rating').fillna(0)

In [ ]:
rating_mt_b.reset_index(inplace=True)
rating_mt_b = rating_mt_b.rename_axis(None, axis=1)
rating_mt_b.head()

In [ ]:
# Create a place holder matrix for similarities, and fill in the user name column
data_sims = pd.DataFrame(index=rating_mt_b.index,columns=rating_mt_b.columns)

In [ ]:
data_sims

In [ ]:
data_sims.iloc[:,:1] = rating_mt_b.iloc[:,:1]

In [ ]:
data_sims

In [ ]:
# Create a placeholder items for closes neighbours to an item
data_neighbours = pd.DataFrame(index=data_mv.columns,columns=range(1,12))

In [ ]:
data_neighbours

In [ ]:
#Loop through all rows, skip the user column, and fill with similarity scores
for i in range(0,len(data_sims.index)):
    for j in range(1,len(data_sims.columns)):
        user = data_sims.index[i]
        product = data_sims.columns[j]
 
        if rating_mt_b.iloc[i][j] == 1:
            data_sims.iloc[i][j] = 0
        else:
            # product_top_names = data_neighbours[product].iloc[1:11]
            # print(product_top_names)
            # product_top_sims = data_mv[product].iloc.sort_values(ascending=False)[1:11]
            # user_purchases = rating_mt.iloc[user,product_top_names]
 
            data_sims.iloc[i][j] = getScore(user_purchases,product_top_sims)

In [ ]:
i = 1
while i > 0 :
  # print(i)
  if i == 1000000000000000000000000000000000000000000000:
    break
  i += 1